In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1-Sphere Eigenfunctions (S¹) parameters ---
max_k = 6
# Build labels: constant, then cos(kθ), sin(kθ)
eigen_labels = (
    ["const (k=0)"] +
    [f"cos({k}θ)" for k in range(1, max_k+1)] +
    [f"sin({k}θ)" for k in range(1, max_k+1)]
)
num_eigen = len(eigen_labels)

# --- Grid on the circle S¹ ---
n_theta = 1000
theta   = np.linspace(0, 2*np.pi, n_theta, endpoint=False)
dtheta  = theta[1] - theta[0]

# --- Uniform density ρ = 1/(2π) ---
rho = np.ones_like(theta) / (2*np.pi)

# --- Storage for f, |∇f|², and eigenvalues λ ---
f_list       = []
grad2_list   = []
lambda_list  = []

# --- k=0 mode (constant eigenfunction) ---
f0 = np.ones_like(theta)
norm0 = np.sqrt(np.sum(f0**2 * rho * dtheta))
f0 /= norm0
df0 = np.gradient(f0, dtheta)
f_list.append(f0)
grad2_list.append(df0**2)
lambda_list.append(0)

# --- k ≥ 1 modes: cos(kθ), sin(kθ) with eigenvalue k² ---
for k in range(1, max_k+1):
    for mode in ("cos", "sin"):
        if mode == "cos":
            f = np.cos(k * theta)
        else:
            f = np.sin(k * theta)
        # normalize in L²(ρ dθ)
        norm_f = np.sqrt(np.sum(f**2 * rho * dtheta))
        f /= norm_f

        # gradient and its square
        df = np.gradient(f, dtheta)
        grad2 = df**2

        f_list.append(f)
        grad2_list.append(grad2)
        lambda_list.append(k**2)

# --- Compute σ_eta for each eigenfunction ---
sigma_eta = []
for i, λ in enumerate(lambda_list):
    if λ == 0:
        sigma_eta.append(0.0)
    else:
        num = 2 * λ
        den = np.sum(grad2_list[i] * rho**2 * dtheta)
        sigma_eta.append(num / den)

# --- Build variance–covariance matrix ---
variance_matrix = np.zeros((num_eigen, num_eigen))
for i in range(num_eigen):
    term_i = (
        lambda_list[i]
        + lambda_list[i] * f_list[i]**2
        - sigma_eta[i] * grad2_list[i] * rho
    )
    for j in range(num_eigen):
        term_j = (
            lambda_list[j]
            + lambda_list[j] * f_list[j]**2
            - sigma_eta[j] * grad2_list[j] * rho
        )
        variance_matrix[i, j] = np.sum(term_i * term_j * rho * dtheta)

# --- Visualize covariance heatmap on S¹ ---
plt.figure(figsize=(10, 8))
sns.heatmap(
    variance_matrix,
    cmap="coolwarm",
    square=True,
    xticklabels=eigen_labels,
    yticklabels=eigen_labels,
    cbar_kws={"label": "Covariance"}
)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.title("Variance–Covariance Matrix for Laplace–Beltrami on S¹")
plt.tight_layout()
plt.show()
